In [5]:
import pandas as pd
import os
from datetime import datetime, time  # <-- We add 'time' here

# ----------------------------------------------------------------------
# 1) PATH SETUP
# ----------------------------------------------------------------------
# Change this to wherever your SP500_2010_extracted folder is:
base_dir = "SP500_2010_extracted"

# Define market open and close times
MARKET_OPEN  = time(9, 30)
MARKET_CLOSE = time(16, 0)

# ----------------------------------------------------------------------
# 2) HELPER FUNCTIONS
# ----------------------------------------------------------------------
def load_bbo_data(file_path):
    """
    Reads a CSV where we skip the first line (if malformed) and assume
    each subsequent line has columns:
        [bid_price, bid_size, ask_price, ask_size, timestamp]
    Then we convert 'timestamp' to datetime, *filter out pre/post market*,
    and sort.
    """
    df = pd.read_csv(
        file_path,
        skiprows=1,  # <-- Skip first line if it's malformed
        header=None,
        names=["bid_price", "bid_size", "ask_price", "ask_size", "timestamp"],
        on_bad_lines="skip",  # skip any weird lines
    )
    
    # Convert to actual datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    
    # Drop rows where timestamp couldn't be parsed
    df.dropna(subset=["timestamp"], inplace=True)
    
    # Filter out times before 09:30 and after 16:00
    df = df[df["timestamp"].dt.time.between(MARKET_OPEN, MARKET_CLOSE)]
    
    df.sort_values("timestamp", inplace=True)
    return df

def load_trade_data(file_path):
    """
    Reads a CSV that *has a header row* with columns, e.g.:
       [trade-price, trade-volume, trade-stringflag, trade-rawflag, ny_timestamp]
    Then we keep only relevant columns, *filter out pre/post market*,
    and sort.
    """
    df = pd.read_csv(file_path, header=0)
    
    df["timestamp"] = pd.to_datetime(df["ny_timestamp"], errors="coerce")
    # Drop rows where timestamp couldn't be parsed
    df.dropna(subset=["timestamp"], inplace=True)

    # Filter out times before 09:30 and after 16:00
    df = df[df["timestamp"].dt.time.between(MARKET_OPEN, MARKET_CLOSE)]
    
    # Keep only the columns we need
    df = df[["trade-price", "trade-volume", "timestamp"]]
    # Rename them
    df.columns = ["trade_price", "trade_volume", "timestamp"]

    df.sort_values("timestamp", inplace=True)
    return df

def calculate_ofi(df):
    """
    Calculate Order Flow Imbalance (OFI):
       ofi_t = (bid_price_t - bid_price_{t-1}) * bid_size_t
               - (ask_price_t - ask_price_{t-1}) * ask_size_t
    """
    df["delta_bid"] = df["bid_price"].diff()
    df["delta_ask"] = df["ask_price"].diff()

    df["ofi"] = (df["delta_bid"] * df["bid_size"]) - (df["delta_ask"] * df["ask_size"])
    return df

# ----------------------------------------------------------------------
# 3) MAIN SCRIPT
# ----------------------------------------------------------------------
if __name__ == "__main__":

    # Loop over each stock folder, e.g. AAP_05, AA_05, etc.
    for stock_folder in os.listdir(base_dir):
        stock_path = os.path.join(base_dir, stock_folder)
        
        # Skip if it's not actually a folder
        if not os.path.isdir(stock_path):
            continue

        print(f"Processing stock folder: {stock_folder}")
        
        # Subfolders where BBO and trade files live
        bbo_dir = os.path.join(stock_path, "bbo")
        trade_dir = os.path.join(stock_path, "trade")

        # If either subfolder doesn't exist, skip
        if not (os.path.isdir(bbo_dir) and os.path.isdir(trade_dir)):
            print(f"  Skipping {stock_folder}: 'bbo' or 'trade' folder not found.")
            continue
        
        # Make an output folder for processed data
        processed_dir = os.path.join(stock_path, "processed")
        os.makedirs(processed_dir, exist_ok=True)

        # Go through each BBO file, e.g. 2010-05-06-AAP-bbo.csv
        bbo_files = sorted(f for f in os.listdir(bbo_dir) if f.endswith("-bbo.csv"))
        
        for bbo_file in bbo_files:
            full_bbo_path = os.path.join(bbo_dir, bbo_file)

            # Split by '-' to parse date + ticker
            parts = bbo_file.split("-")  
            if len(parts) < 5:
                # Something unexpected in the filename
                continue

            date_str = "-".join(parts[0:3])   # e.g. "2010-05-06"
            ticker   = parts[3]              # e.g. "AAP"

            # Build matching trade filename: 2010-05-06-AAP-trade.csv
            trade_file      = f"{date_str}-{ticker}-trade.csv"
            full_trade_path = os.path.join(trade_dir, trade_file)

            # If the trade file doesn't exist, skip
            if not os.path.exists(full_trade_path):
                continue

            # Load data
            bbo_df   = load_bbo_data(full_bbo_path)
            trade_df = load_trade_data(full_trade_path)

            if bbo_df.empty or trade_df.empty:
                print(f"  No data in {bbo_file} or {trade_file}. Skipping.")
                continue

            # Merge them as-of (nearest previous match)
            merged_df = pd.merge_asof(
                trade_df.sort_values("timestamp"),
                bbo_df.sort_values("timestamp"),
                on="timestamp",
                direction="backward"
            )

            # Calculate mid_price, spread
            merged_df["mid_price"] = (merged_df["bid_price"] + merged_df["ask_price"]) / 2
            merged_df["spread"]    = merged_df["ask_price"] - merged_df["bid_price"]

            # Calculate OFI
            merged_df = calculate_ofi(merged_df)
            # Drop first row if it has NaN due to diff()
            merged_df.dropna(subset=["ofi"], inplace=True)

            # Save result
            output_file = os.path.join(processed_dir, f"{date_str}-{ticker}-processed.csv")
            merged_df.to_csv(output_file, index=False)
            
            print(f"  Processed {bbo_file} => {output_file}")

            # If you truly don't need the original raw files:
            # os.remove(full_bbo_path)
            # os.remove(full_trade_path)

    print("All done!")


Processing stock folder: HSY_05
  Processed 2010-05-03-HSY-bbo.csv => SP500_2010_extracted/HSY_05/processed/2010-05-03-HSY-processed.csv
  Processed 2010-05-04-HSY-bbo.csv => SP500_2010_extracted/HSY_05/processed/2010-05-04-HSY-processed.csv
  Processed 2010-05-05-HSY-bbo.csv => SP500_2010_extracted/HSY_05/processed/2010-05-05-HSY-processed.csv
  Processed 2010-05-06-HSY-bbo.csv => SP500_2010_extracted/HSY_05/processed/2010-05-06-HSY-processed.csv
  Processed 2010-05-07-HSY-bbo.csv => SP500_2010_extracted/HSY_05/processed/2010-05-07-HSY-processed.csv
  Processed 2010-05-10-HSY-bbo.csv => SP500_2010_extracted/HSY_05/processed/2010-05-10-HSY-processed.csv
  Processed 2010-05-11-HSY-bbo.csv => SP500_2010_extracted/HSY_05/processed/2010-05-11-HSY-processed.csv
  Processed 2010-05-12-HSY-bbo.csv => SP500_2010_extracted/HSY_05/processed/2010-05-12-HSY-processed.csv
  Processed 2010-05-13-HSY-bbo.csv => SP500_2010_extracted/HSY_05/processed/2010-05-13-HSY-processed.csv
  Processed 2010-05-14-